In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import copy

In [2]:
# --- MCTS Node Class ---
class MCTSNode:
    def __init__(self, state, parent=None, action=None, make_env=None, verbose=False):
        self.state = state
        self.parent = parent
        self.action = action
        self.children = []
        self.visits = 0
        self.value = 0.0
        self.make_env = make_env
        self.verbose = verbose

    def is_leaf(self):
        return len(self.children) == 0

    def uct(self, exploration=1.41):
        if self.visits == 0 or self.parent is None:
            return float("inf")
        exploitation = self.value / self.visits
        exploration_bonus = exploration * math.sqrt(math.log(self.parent.visits) / self.visits)
        return exploitation + exploration_bonus

    def best_uct_child(self):
        return max(self.children, key=lambda child: child.uct())

    def best_child(self):
        return max(self.children, key=lambda child: child.value)

    def selection(self, env):
        """Traverse the tree to select a promising node to expand."""
        node = self
        while not node.is_leaf():
            node = node.best_uct_child()
        if node.visits == 0:
            return node, None
        goal_node = node.expand(env)
        return (goal_node if goal_node else random.choice(node.children), goal_node is not None)

    def expand(self, env):
        """Expand the node by trying all possible actions."""
        for action in range(env.action_space.n):
            env_copy = self.make_env()
            env_copy.reset()
            env_copy.unwrapped.s = self.state
            obs, reward, terminated, truncated, _ = env_copy.step(action)

            if reward == 0 and terminated or self.state == obs:
                continue

            child = MCTSNode(obs, parent=self, action=action, make_env=self.make_env, verbose=self.verbose)
            self.children.append(child)

            if reward == 1:
                if self.verbose:
                    print(f"Goal found from state {self.state} with action {action} → {obs}")
                return child

        if self.verbose:
            print(f"Expanded node {self.state} with children: {[c.state for c in self.children]}")
        return None

    def simulation(self, max_steps=10):
        """Perform a rollout from the current node using random actions."""
        env_copy = self.make_env()
        env_copy.reset()
        env_copy.unwrapped.s = self.state
        obs = self.state

        for _ in range(max_steps):
            action = env_copy.action_space.sample()
            obs, reward, terminated, truncated, _ = env_copy.step(action)

            if self.verbose:
                print(f"Simulating from {self.state} → {obs} with action {action} reward {reward}")

            if reward == 1 or terminated or truncated:
                return reward
        return 0

    def backpropagation(self, reward):
        """Propagate the simulation result back up the tree."""
        node = self
        while node:
            node.visits += 1
            node.value += reward
            if self.verbose:
                print(f"Backprop node {node.state}, visits={node.visits}, value={node.value}")
            node = node.parent


# --- MCTS Search Class ---
class MCTS:
    def __init__(self, make_env, num_iterations=100, num_simulations=10, exploration=1.41, verbose=False):
        self.make_env = make_env
        self.env = make_env()
        self.root = MCTSNode(self.env.reset()[0], make_env=self.make_env, verbose=verbose)
        self.num_iterations = num_iterations
        self.num_simulations = num_simulations
        self.exploration = exploration
        self.verbose = verbose
        self.root.expand(self.env)

    def run(self):
        for _ in range(self.num_iterations):
            node, goal = self.root.selection(self.env)
            if goal:
                node.backpropagation(1)
                if self.verbose:
                    print(f"Goal reached at state {node.state}")
                break
            reward = node.simulation(max_steps=self.num_simulations)
            node.backpropagation(reward)
        if self.verbose:
            print(f"Finished {self.num_iterations} iterations.")

In [3]:
class ManipulationEnv(gym.Env):
    """Custom GridWorld environment with an agent and movable box."""
    
    def __init__(self, grid_size=4, agent_start=0, box_start=1, goal_agent=13, goal_box=15, holes=None):
        super().__init__()
        self.grid_size = grid_size
        self.n_states = grid_size * grid_size
        self.agent_start = agent_start
        self.box_start = box_start
        self.goal_agent = goal_agent
        self.goal_box = goal_box
        self.agent_pos = agent_start
        self.box_pos = box_start
        self.holding = False
        self.holes = holes if holes is not None else {5, 7, 11, 12}

        # Actions: up, down, left, right, grab, release
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.MultiDiscrete([self.n_states, self.n_states, 2])
    
    def reset(self):
        self.agent_pos = self.agent_start
        self.box_pos = self.box_start
        self.holding = False
        return self._get_obs(), {}

    def _get_obs(self):
        return (self.agent_pos, self.box_pos, int(self.holding))

    def _to_coord(self, pos):
        return divmod(pos, self.grid_size)

    def _to_index(self, row, col):
        return row * self.grid_size + col

    def _move(self, pos, action):
        row, col = self._to_coord(pos)
        if action == 0 and row > 0: row -= 1        # up
        elif action == 1 and row < self.grid_size - 1: row += 1  # down
        elif action == 2 and col > 0: col -= 1      # left
        elif action == 3 and col < self.grid_size - 1: col += 1  # right
        return self._to_index(row, col)

    def step(self, action):
        if action in [0, 1, 2, 3]:  # movement
            new_pos = self._move(self.agent_pos, action)
            if new_pos in self.holes:
                return self._get_obs(), 0.0, True, False, {}
            if self.holding:
                self.agent_pos = new_pos
                self.box_pos = new_pos
            else:
                self.agent_pos = new_pos

        elif action == 4:  # grab
            if self.agent_pos == self.box_pos:
                self.holding = True

        elif action == 5:  # release
            self.holding = False

        # If agent or box is in a hole, terminate
        if self.agent_pos in self.holes or self.box_pos in self.holes:
            return self._get_obs(), 0.0, True, False, {}

        # Check goal condition
        success = (
            self.agent_pos == self.goal_agent and
            self.box_pos == self.goal_box and
            not self.holding
        )
        reward = 1.0 if success else 0.0
        return self._get_obs(), reward, success, False, {}


In [5]:

# --- Main Execution ---
def make_env():
    return ManipulationEnv()

# Run MCTS
mcts = MCTS(make_env=make_env, num_iterations=1000, num_simulations=100, exploration=1.41, verbose=False)
mcts.run()

# Visualise best path
env = make_env()
env.reset()
print("Initial state:")

trajectory = []
node = mcts.root
trajectory.append((node.state, node.action, node.value))
while not node.is_leaf():
    prev_node = node
    node = prev_node.best_child()
    trajectory.append((node.state, node.action, node.value))
    print(f"Best action from state {prev_node.state} to state {node.state} with value {node.value}")
    env.step(node.action)

Initial state:
Best action from state (0, 1, 0) to state (4, 1, 0) with value 0.0
Best action from state (4, 1, 0) to state (0, 1, 0) with value 0.0
Best action from state (0, 1, 0) to state (4, 1, 0) with value 0.0
Best action from state (4, 1, 0) to state (0, 1, 0) with value 0.0
Best action from state (0, 1, 0) to state (4, 1, 0) with value 0.0
Best action from state (4, 1, 0) to state (0, 1, 0) with value 0.0


In [4]:
env = ManipulationEnv(goal_agent=2, goal_box=2)
obs, _ = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    prev_obs = obs
    obs, reward, done, truncated, _ = env.step(action)
    
    print(f"Moved from {prev_obs} to {obs} with action {action} and reward {reward}")


Moved from (0, 1, 0) to (0, 1, 0) with action 2 and reward 0.0
Moved from (0, 1, 0) to (0, 1, 0) with action 5 and reward 0.0
Moved from (0, 1, 0) to (1, 1, 0) with action 3 and reward 0.0
Moved from (1, 1, 0) to (1, 1, 1) with action 4 and reward 0.0
Moved from (1, 1, 1) to (1, 1, 1) with action 1 and reward 0.0
